# 第7章：単語ベクトル


In [ ]:
import os
from tqdm import tqdm
from collections import defaultdict

import bhtsne
from gensim.models import KeyedVectors
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import spearmanr
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans

In [ ]:
DATADIR = "data"          # データを保存するおおもとのディレクトリ
CURRENTDIR = "/workspace/notebook" # notebookディレクトリへのパス
CHAPDIR = os.path.join(DATADIR, "chapter7")

try:
    os.mkdir(CHAPDIR)
except:
    print("作成済み等の理由でディレクトリが作成されませんでした")

## 60. 単語ベクトルの読み込みと表示

> Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

### ダウンロード

In [ ]:
# データのダウンロード（割と時間かかる）
!wget -P $CURRENTDIR/$CHAPDIR "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"


### 単語ベクトルの確認

In [ ]:
# モデルの読み込み（割と時間かかる）
model_path = os.path.join(CHAPDIR, "GoogleNews-vectors-negative300.bin.gz")
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
model["United_States"]

## 61. 単語の類似度

> “United States”と”U.S.”のコサイン類似度を計算せよ．

In [ ]:
model.similarity("United_States", "U.S.")

## 62. 類似度の高い単語10件

> “United States”とコサイン類似度が高い10語と，その類似度を出力せよ．

In [ ]:
model.most_similar(positive="United_States")

## 63. 加法構成性によるアナロジー

> “Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [ ]:
result_vec = model["Spain"] - model["Madrid"] + model["Athens"]

model.similar_by_vector(result_vec)

In [ ]:
model.most_similar(positive=["Spain", "Athens"], negative=["Madrid"], topn=10)

出力の違いは↓参考。正規化してるかしていないかの違い？

https://stackoverflow.com/questions/50275623/difference-between-most-similar-and-similar-by-vector-in-gensim-word2vec

## 64. アナロジーデータでの実験

> 単語アナロジーの評価データをダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．

In [ ]:
# データのダウンロード
!wget -P $CURRENTDIR/$CHAPDIR "http://download.tensorflow.org/data/questions-words.txt"

In [ ]:
analogy_fpath = os.path.join(CHAPDIR, "questions-words.txt")
output_fpath = os.path.join(CHAPDIR, "answer.txt")

In [ ]:
if input("実行にめっちゃ時間かかるけどええか？(y/n)") == "y"
    with open(analogy_fpath, "r", encoding="utf8")as fr, open (output_fpath, "w", encoding="utf8")as fw:
        # 各行読み込み
        for line in tqdm(fr.readlines()):
            if line.startswith(":"):
                # カテゴリ行はそのまま出力
                fw.write(line)
            else:
                # 単語の組み合わせの場合
                words = line.rstrip("\n").split(" ")
                sim_word, cos = model.most_similar(positive=[words[1], words[2]], 
                                                   negative=[words[0]], topn=1)[0]
                fw.write(" ".join([line.rstrip("\n"), sim_word, str(cos)]))
                fw.write("\n")


## 65. アナロジータスクでの正解率

> 64の実行結果を用い，意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ．

In [ ]:
# データの読み込み

sem_tf_list = []
syn_tf_list = []

with open(output_fpath, "r", encoding="utf8")as fr:
    flag = "sem"
    for line in fr:
        # 文法的アナロジーのところ
        if line.startswith(":"):
            if line.startswith(": gram"):
                flag = "syn"
        else:
            sep_line = line.rstrip("\n").split(" ")
            if flag == "sem":
                sem_tf_list.append(sep_line[3] == sep_line[4])
            elif flag == "syn":
                syn_tf_list.append(sep_line[3] == sep_line[4])

In [ ]:
print(f"意味的アナロジー正解率：{sem_tf_list.count(True)/len(sem_tf_list):.3f}")
print(f"文法的アナロジー正解率：{syn_tf_list.count(True)/len(syn_tf_list):.3f}")

## 66. WordSimilarity-353での評価Permalink

> The WordSimilarity-353 Test Collectionの評価データをダウンロードし，単語ベクトルにより計算される類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

### データのダウンロード

In [ ]:
# 保存ディレクトリへ移動
%cd $CURRENTDIR/$CHAPDIR
# ダウンロードと解凍
!wget http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
!unzip wordsim353.zip
!head -5 './combined.csv'
# 作業ディレクトリに戻す
%cd $CURRENTDIR

In [ ]:
# 単語ベクトルで類似度算出
fpath = os.path.join(CHAPDIR, "combined.csv")

result = [] # データ全体用
with open(fpath, "r", encoding="utf8")as fr:
    # 読み飛ばし
    next(fr)    
    for line in fr:
        word1, word2, human_score = line.rstrip("\n").split(",")
        sim = model.similarity(word1, word2)
        result.append([word1, word2, float(human_score), sim])
    

In [ ]:
# スピアマン相関係数を算出
human_scores= np.array(result).T[2]
w2v_sims = np.array(result).T[3]

result = spearmanr(human_scores, w2v_sims)
result

## 67. k-meansクラスタリング

> 国名に関する単語ベクトルを抽出し，k-meansクラスタリングをクラスタ数k=5として実行せよ．



In [ ]:
input_fpath = os.path.join(CHAPDIR, "questions-words.txt")

# 国名の取得
countries = set()
with open(input_fpath, "r", encoding="utf8") as f:
    mode = None
    for line in f:
        if "capital-common-countries" in line or "capital-world" in line:
            mode = 1
            continue
        elif "currency" in line or "gram6-nationality-adjective" in line:
            mode =0
            continue
        elif line.startswith(":"):
            mode = None
            continue
        if mode:
            countries.add(line.split(" ")[mode])
countries = list(countries)

# 単語ベクトルの取得
countries_vec = [model[country] for country in countries]

In [ ]:
countries[:5]

In [ ]:
countries_vec

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(countries_vec)

In [ ]:
clusters = defaultdict(lambda:list())

for label, countrie in zip(kmeans.labels_, countries):
    clusters[label].append(countrie)

# 表示
for label in range(5):
    print(f"■■■ cluster-{label} ■■■")
    print(", ".join(clusters[label]))

## 68. Ward法によるクラスタリングPermalink

> 国名に関する単語ベクトルに対し，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

In [ ]:
plt.figure(figsize=(15, 8))
Z = linkage(countries_vec, method="ward")
dendrogram(Z, labels=countries)
plt.show()

## 69. t-SNEによる可視化

In [ ]:
embedded = bhtsne.tsne(np.array(countries_vec).astype(np.float64), dimensions=2, rand_seed=123)
plt.figure(figsize=(16, 9))
plt.scatter(np.array(embedded).T[0], np.array(embedded).T[1])
for (x, y), name in zip(embedded, countries):
    plt.annotate(name, (x, y))
plt.show()